generate conffile

In [5]:
multnamearr = range(10,91,20)
multstrarr = ["0 1 15","0 1 5","0 1 3","0 7 15","0 3 5"]
rtnamearr = ["ecmp","su2"]
rt1arr = ["ecmp","su"]
rt2arr = [0,2]
pwnamearr = ["equal","thisweight"]
fseedarr = range(10)

for imult,multname in enumerate(multnamearr):
    multstr = multstrarr[imult]

    lfarr = [2,20,204]
    with open(f"/home/annzhou/DRing/src/emp/datacentre/experiments/linkfailure/leafspine_{multname}.conf",'w') as f:
        rtname = "ecmp"
        rt1 = "ecmp"
        rt2 = 0
        pwname = "equal"
        pwfile = f"pathweightfiles/leafspine/{rtname}/pathweight_leafspine_{rtname}_equal_64.txt"

        f.write(f"./run.sh LEAFSPINE MAKE 80 3072 64 leafspine_a2a A2A null {multstr} 0 0 0 0 0 0 {rt1} {rt2} null 0 netpathfiles/netpath_{rtname}_leafspine.txt {pwfile} 64 50 150 200 0 0 > m_linkfailure_leafspine_{rtname}_a2a_{pwname}_{multname}_0_0.log\n")
        for lf in lfarr:
            for fseed in fseedarr:
                f.write(f"./run.sh LEAFSPINE NOMAKE 80 3072 64 leafspine_a2a A2A null {multstr} 0 0 0 0 0 0 {rt1} {rt2} null 0 netpathfiles/netpath_{rtname}_leafspine.txt {pwfile} 64 50 150 200 {lf} {fseed} > m_linkfailure_leafspine_{rtname}_a2a_{pwname}_{multname}_{lf}_{fseed}.log\n")

    lfarr = [2,21,213]
    with open(f"/home/annzhou/DRing/src/emp/datacentre/experiments/linkfailure/dring_{multname}.conf",'w') as f:
        make_dring = True
        for irtname,rtname in enumerate(rtnamearr):
            rt1 = rt1arr[irtname]
            rt2 = rt2arr[irtname]
            for pwname in pwnamearr:
                pwfile = f"pathweightfiles/dring/{rtname}/a2a/pathweight_pbr1_dring_{rtname}_a2a_lp1_barriernocrossover_64.txt" if pwname=="thisweight" else f"pathweightfiles/dring/{rtname}/pathweight_dring_{rtname}_equal_64.txt"
                make = "MAKE" if make_dring else "NOMAKE"
                f.write(f"./run.sh RRG {make} 80 2988 64 dring_a2a A2A graphfiles/ring_supergraph/double_ring/instance1_80_64.edgelist {multstr} 0 0 0 0 0 0 {rt1} {rt2} null 0 netpathfiles/netpath_{rtname}_dring.txt {pwfile} 64 50 150 200 0 0 > m_dring_{rtname}_a2a_{pwname}_{multname}_0_0.log\n")
                make_dring = False

                for lf in lfarr:
                    for fseed in fseedarr:
                        f.write(f"./run.sh RRG NOMAKE 80 2988 64 dring_a2a A2A graphfiles/ring_supergraph/double_ring/instance1_80_64.edgelist {multstr} 0 0 0 0 0 0 {rt1} {rt2} null 0 netpathfiles/netpath_{rtname}_dring.txt {pwfile} 64 50 150 200 {lf} {fseed} > m_dring_{rtname}_a2a_{pwname}_{multname}_{lf}_{fseed}.log\n")

parse test conf

In [3]:
# test
homedir = "/home/annzhou/"
conffile = f"{homedir}DRing/src/emp/datacentre/experiments/linkfailure/test.conf"
sumfile = f"{homedir}DRing/src/emp/datacentre/experiments/linkfailure/test.txt"
datadict = dict()
with open(conffile,'r') as f:
    with open(sumfile,'a') as sumf:
        lines = f.readlines()
        for line in lines:
            tokens = line.split()
            logfile = tokens[31]
            sizelist = list()
            durationlist = list()
            startlist = list()
            with open(f"{homedir}DRing/src/emp/datacentre/{logfile}",'r') as logf:
                loglines = logf.readlines()
                for logline in loglines:
                    logtokens = logline.split()
                    if logtokens[0] == "FCT":
                        durationlist.append(float(logtokens[2]))
                        sizelist.append(int(logtokens[1]))
                        startlist.append(float(logtokens[3]))
            datadict[logfile] = [durationlist,sizelist,startlist]
            durationlist.sort()
            sumf.write(f"{logfile}\t{sum(durationlist)/len(durationlist)}\t{durationlist[int(len(durationlist)*0.5)]}\t{durationlist[int(len(durationlist)*0.99)]}\t{durationlist[int(len(durationlist)*0.9999)]}\n")